# Несколько временных интервалов
Пример расчета показателей телеканала по временным интервалам.

## Описание задачи и условий расчета
- Период: 01.08.2022 - 31.08.2022
- Временной интервал: понедельник-пятница 19:00-23:00 и суббота-воскресенье 10:00-23:00
- ЦА: Все 10-45
- Место просмотра: Все места (дом+дача)
- Каналы: СТС									
- Статистики: Share

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

In [ ]:
# выберем телесети по имени (части имени)
cats.get_tv_net(name=["СТС"])

# далее в запросах будем использовать id телесетей.

In [ ]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['возраст'])

In [ ]:
# найдем номера нужных дней недели
cats.get_tv_day_week()

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-08-01', '2022-08-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = 'age >= 10 AND age <= 45'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId IN (11)'

# Указываем список статистик для расчета
statistics = ['Share', 'Rtg000']

# Указываем срезы
slices = ['tvNetName']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Рассмотрим возможности выбора временного интервала

In [ ]:
# выбор полных суток (указываем полный диапазон или None)
# time_filter = 'timeBand1 >= 50000 AND timeBand1 < 290000'

# выбор слота с детализацией до минуты. В примере с 18 до 22 часов
# time_filter = 'timeBand1 >= 180000 AND timeBand1 < 220000'

# выбор комбинации день недели + слот (например, понедельник-пятница 19:00-23:00 + суббота-воскресенье 10:00-23:00)
# time_filter = 'timeBand1 >= 190000 AND timeBand1 < 230000'
# И одновременно задаем фильтр дня недели - с понедельника по пятницу
# weekday_filter = 'researchWeekDay IN (1, 2, 3, 4, 5)'

Зададим группы требуемых временных интервалов с днями недели

In [ ]:
# Задаем необходимые группы
groups = {
    'понедельник-пятница 19:00-23:00':{
        "timeband":'timeBand1 >= 190000 AND timeBand1 < 230000',
        "weekday": 'researchWeekDay IN (1, 2, 3, 4, 5)'
    },
    'суббота-воскресенье 10:00-23:00':{
        "timeband":'timeBand1 >= 100000 AND timeBand1 < 230000',
        "weekday": 'researchWeekDay IN (6, 7)'
    }    
}

## Расчет задания

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой группы формируем задания и отправляем на расчет
for group, logical in groups.items():
    
    # Формируем задание на расчет ЦА
    project_name = f"{group}" 
    time_filter = f"{logical['timeband']}"
    weekday_filter = f"{logical['weekday']}"
      
    # Формируем из указанных параметров задание для TVI API в формате JSON для каждого из этапов цикла
    task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)
df

Настраиваем внешний вид таблицы

In [ ]:
df_res = pd.pivot_table(df, values=['Share'],
                        index=['prj_name'], 
                        columns=['tvNetName'])
df_res

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('05_complex_timeband')) as writer:
    df_res.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)